# 経済産業省の通商白書を全文検索するAPIサーバの計画

Date: 2024/04/29-

注意：汎用化し、経済産業省以外の省庁が発行する白書も検索可能にする。

## 前処理
- Beautiful SoapでPDFファイルのリンクリスト作成
- PyMuPDFで各PDFファイルのText抽出しSQLiteへ格納
- 同時にspaCyでNERを行い、その結果をSQLiteへ格納

```
Table "source" <= 経産省、総務省、XX株式会社といった白書やIR資料を提供している団体名
base_url(primary key), org

Table "links" <= Beautiful Soapで抽出したPDFファイルのURl
path(primary key), title, fk(source:base_url)

Table "texts" <= PyMuPDFで抽出したテキスト
fk(links:path), page, text

Table "named_entities" <= spaCyで抽出したNamed Entity
fk(texts:path), page, named_entity, label
```

## APIサーバ

#### /contain?keywords=\<keywords\>

\[{base_url: \<base_url\>, path: \<path\>, pages: \<pages\>},...\]

#### /highlight/\<base_url\>/\<path\>/\<page\>?\<keywords\>

該当キーワードをハイライトしてPDFを返送。keywordsなしのときはハイライトせず返信。
指定ページの前後で合計３ページ分を返信。

## Table: source

In [1]:
import sqlite3

base_url_meti = "https://www.meti.go.jp"
org_meti = "経済産業省"
doc = "通商白書"

DB_PATH = 'database/search.db'

with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS source')
    cur.execute('CREATE TABLE source (base_url TEXT PRIMARY KEY, org TEXT, doc TEXT, UNIQUE(org, doc))')
    cur.execute(f'INSERT INTO source (base_url, org, doc) VALUES ("{base_url_meti}", "{org_meti}", "{doc}")')

## Table: links

PDFリンク抽出

In [2]:
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    base_url = cur.execute('SELECT base_url FROM source WHERE org = "経済産業省" AND doc="通商白書"').fetchone()[0]

base_url

'https://www.meti.go.jp'

In [3]:
url_meti = "https://www.meti.go.jp/report/tsuhaku2023/whitepaper_2023.html"

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
resp = requests.get(url_meti)
html_doc = resp.content.decode('utf-8')

In [6]:
soup = BeautifulSoup(html_doc, 'html.parser')

In [7]:
chapter1 = [tag for tag in soup.find_all(string='第Ⅰ部　岐路に立たされる世界経済')]

In [8]:
chapter1[0].find_all_next("ul", {"class": "lnkLst"})

[<ul class="lnkLst">
 <li>
 <a href="/report/tsuhaku2023/pdf/1-1-1.pdf">第1節　世界経済の現状と見通し（PDF形式：3,448KB）<img alt="PDFファイル" height="16" src="/img_2017/template/icon_pdf.gif" width="16"/></a></li>
 <li>
 <a href="/report/tsuhaku2023/pdf/1-1-2.pdf">第2節　ロシアによるウクライナ侵略を巡る状況とその影響（PDF形式：4,379KB）<img alt="PDFファイル" height="16" src="/img_2017/template/icon_pdf.gif" width="16"/></a></li>
 <li>
 <a href="/report/tsuhaku2023/pdf/1-1-3.pdf">第3節　高まるインフレ圧力（PDF形式：4,415KB）<img alt="PDFファイル" height="16" src="/img_2017/template/icon_pdf.gif" width="16"/></a></li>
 <li>
 <a href="/report/tsuhaku2023/pdf/1-1-4.pdf">第4節　新興国・途上国で高まる債務リスク（PDF形式：4,771KB）<img alt="PDFファイル" height="16" src="/img_2017/template/icon_pdf.gif" width="16"/></a></li>
 <li>
 <a href="/report/tsuhaku2023/pdf/1-1-c1.pdf">コラム1　1980年代の中南米の累積債務問題（PDF形式：1,776KB）<img alt="PDFファイル" height="16" src="/img_2017/template/icon_pdf.gif" width="16"/></a></li>
 <li>
 <a href="/report/tsuhaku2023/pdf/1-1-5.pdf">第5節　分断の危機に直面する世界経済（PDF形式：3,758KB）<img alt="PD

In [9]:
all_ul = chapter1[0].find_all_next("ul", {"class": "lnkLst"})

all_as = []
for ul in all_ul:
    #print(ul.find_all('a'))
    all_as.extend(ul.find_all('a'))

# [[url, title], ...]
links = filter(lambda x: x[0].endswith('.pdf') ,[[a['href'], a.text] for a in all_as])

In [10]:
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS links')
    cur.execute('CREATE TABLE links (path TEXT PRIMARY KEY, title TEXT, base_url TEXT, FOREIGN KEY(base_url) REFERENCES source(base_url))')

    for path, title in links:
        cur.execute(f'INSERT INTO links (path, title, base_url) VALUES(?, ?, ?)', (path, title, base_url))

## Table: texts

In [11]:
from urllib.parse import urljoin

with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    links = cur.execute('SELECT * FROM links').fetchall()
    
all_url = [urljoin(e[2], e[0]) for e in links]

In [12]:
import requests
import fitz

resp = requests.get(all_url[0])
doc = fitz.open(stream=resp.content)

In [13]:
extract_text = lambda page: (page.number, page.get_text("text").replace('\n', ''))

[extract_text(page) for page in doc]

[(0, '岐路に立たされる世界経済第Ⅰ部'),
 (1, ''),
 (2,
  '第１節 \x07世界経済の現状と見通し第２節 \x07ロシアによるウクライナ侵略を巡る状況とその影響第３節 \x07高まるインフレ圧力第４節 新興国・途上国で高まる債務リスク第５節 分断の危機に直面する世界経済第１章減速感を強める世界経済'),
 (3,
  '\u3000本章では、はじめに、世界経済は、ロシアによるウクライナ侵略による不確実性の高まりやインフレの高進、金融引締めの加速により減速感を強めていること、欧米を中心とした急速な金融引締めは、通貨価値の下落、金利上昇を通じてグローバル・サウスを中心に債務リスクを高めていることを示す。\u3000つぎに、権威主義国の台頭により、世界経済は今、分断の危機に直面する一方、グローバル・サウスは中立的立場で自国の利益を確保する構図となっていることを示す。\u3000最後に、近年、経済依存関係を武器化する経済的威圧に係る事案が増加しており、 WTO の上級委員会が機能不全に陥る中、欧米諸国では対応の検討を加速していることを示す。世界経済の現状と見通し第１節（1）2022 年の世界経済の概要\u30002022 年の世界経済は成長鈍化の1 年であった。世界経済の成長率は、2021 年にはパンデミックからの回復により大幅な上昇を見せたものの、2022 年には成長に落ち着きが見られた。成長鈍化の主な要因としては、ロシアによるウクライナ侵略、インフレの加速、中国経済の成長鈍化があげられる。ロシアによるウクライナ侵略は、天然ガスや原油といった資源価格等の高騰、サプライチェーンの断絶とそれに伴う波及的な効果を発生させ、世界経済の成長にとってマイナスに寄与した。インフレ率の高まりは、パンデミック後の需要回復に伴い2021 年の後半から見られ、ロシアによるウクライナ侵略による資源価格の高騰や供給減少がインフレ率の高まりに拍車をかけた。インフレ率の高まりと、それに対処するための各国中央銀行の政策金利の急速な引上げは、家計の購買力の低下や経済活動の停滞、債務の持続可能性に影響を与え、世界経済の成長鈍化の要因となっている。中国経済は、パンデミックの再拡大と都市封鎖の影響により成長が鈍化し、それが世界経済の成長鈍化にも影響を与えた。\u30002022 年1 

In [18]:
# 本番処理
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS texts')
    cur.execute('CREATE TABLE texts (path TEXT, page INTEGER, text TEXT, FOREIGN KEY(path) REFERENCES links(path))')

    total = len(links)
    cnt = 1
    
    for link in links:
        print(f'{cnt}/{total}', end=' ')
        cnt += 1
        path = link[0]
        url = urljoin(link[2], link[0])
        resp = requests.get(url)
        doc = fitz.open(stream=resp.content)
        for page in doc:
            page_num, text = extract_text(page)
            cur.execute('INSERT INTO texts (path, page, text) VALUES (?, ?, ?)', (path, page_num, text))

1/41, 2/41, 3/41, 4/41, 5/41, 6/41, 7/41, 8/41, 9/41, 10/41, 11/41, 12/41, 13/41, 14/41, 15/41, 16/41, 17/41, 18/41, 19/41, 20/41, 21/41, 22/41, 23/41, 24/41, 25/41, 26/41, 27/41, 28/41, 29/41, 30/41, 31/41, 32/41, 33/41, 34/41, 35/41, 36/41, 37/41, 38/41, 39/41, 40/41, 41/41, 

## Table: named_entities